# Variety Prediction Model

All the markdowns are written in <font color=#a83969>Wine color</font>

<font color=#a83969>During my online research, I discovered that the highest model accuracy achieved on this dataset was 50.48% and 52.12%. However, I was able to achieve highest accuracy of 56.27% using the random forest classifier</font>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df=pd.read_csv("E:\\Assignment_Wine_data\\OSX_DS_assignment.csv")
df.head()

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,@kerinokeefe,Italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia,White Blend
1,@vossroger,Portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos,Portuguese Red
2,@paulgwine,US,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm,Pinot Gris
3,NaN,US,St. Julian 2013 Reserve Late Harvest Riesling ...,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian,Riesling
4,@paulgwine,US,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks,Pinot Noir


In [3]:
df.shape

(103322, 12)

In [4]:
df.groupby('variety')['variety'].agg('count')

variety
Bordeaux-style Red Blend       6915
Bordeaux-style White Blend     1066
Cabernet Franc                 1353
Cabernet Sauvignon             9472
Champagne Blend                1396
Chardonnay                    11753
Gamay                          1025
Gewürztraminer                 1012
Grüner Veltliner               1345
Malbec                         2652
Merlot                         3102
Nebbiolo                       2804
Pinot Grigio                   1052
Pinot Gris                     1455
Pinot Noir                    13272
Portuguese Red                 2466
Portuguese White               1159
Red Blend                      8946
Rhône-style Red Blend          1471
Riesling                       5189
Rosé                           3564
Sangiovese                     2707
Sauvignon Blanc                4967
Sparkling Blend                2153
Syrah                          4142
Tempranillo                    1810
White Blend                    2360
Zinfandel           

<font color=#a83969>First i removed rows user_name, review title, review description, points because they do not concern the model.</font>

In [5]:
df=df.drop(['user_name','review_title','review_description','points'],axis=1)
df.head()

,country,designation,price,province,region_1,region_2,winery,variety
0,Italy,Vulkà Bianco,NaN,Sicily & Sardinia,Etna,NaN,Nicosia,White Blend
1,Portugal,Avidagos,15.0,Douro,NaN,NaN,Quinta dos Avidagos,Portuguese Red
2,US,NaN,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm,Pinot Gris
3,US,Reserve Late Harvest,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian,Riesling
4,US,Vintner's Reserve Wild Child Block,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks,Pinot Noir


<font color=#a83969>Then I attempted to locate the missing values and filled the price column by taking the average of the prices for the corresponding wine variety.</font>

In [6]:
df['price'] = df.groupby('variety')['price'].transform(lambda x: x.fillna(x.mean()).apply(np.floor))

In [7]:
df.isnull().sum()

country           39
designation    29636
price              0
province          39
region_1       16068
region_2       58459
winery             0
variety            0
dtype: int64

<font color=#a83969>From a total of 103322 rows, approximately half of them (58459) did not have any value for region2. As these values do not contribute to the accuracy of the model, I decided to remove them.</font>

In [8]:
df=df.drop(['region_2'],axis=1)
df.head()

,country,designation,price,province,region_1,winery,variety
0,Italy,Vulkà Bianco,23.0,Sicily & Sardinia,Etna,Nicosia,White Blend
1,Portugal,Avidagos,15.0,Douro,NaN,Quinta dos Avidagos,Portuguese Red
2,US,NaN,14.0,Oregon,Willamette Valley,Rainstorm,Pinot Gris
3,US,Reserve Late Harvest,13.0,Michigan,Lake Michigan Shore,St. Julian,Riesling
4,US,Vintner's Reserve Wild Child Block,65.0,Oregon,Willamette Valley,Sweet Cheeks,Pinot Noir


In [9]:
df.isnull().sum()

country           39
designation    29636
price              0
province          39
region_1       16068
winery             0
variety            0
dtype: int64

<font color=#a83969>Similarly, there were 30309 unique values for the "designation" feature, out of which 29636 were empty, making the combined count of empty values and unique values more than half of the dataset (59945). Hence, this feature was also dropped.</font>

In [10]:
df=df.drop(['designation'],axis=1)
df.head()

,country,price,province,region_1,winery,variety
0,Italy,23.0,Sicily & Sardinia,Etna,Nicosia,White Blend
1,Portugal,15.0,Douro,NaN,Quinta dos Avidagos,Portuguese Red
2,US,14.0,Oregon,Willamette Valley,Rainstorm,Pinot Gris
3,US,13.0,Michigan,Lake Michigan Shore,St. Julian,Riesling
4,US,65.0,Oregon,Willamette Valley,Sweet Cheeks,Pinot Noir


In [11]:
df.isnull().sum()

country        39
price           0
province       39
region_1    16068
winery          0
variety         0
dtype: int64

In [12]:
result=df[df['variety']=='White Blend'].groupby('region_1').agg({'region_1':'count'})

most=result.max()[0]
most

108

In [13]:
result=df[df['variety']=='Chardonnay'].groupby('region_1').agg({'region_1':'count'})
result

,region_1
region_1,
Adelaide,2
Adelaide Hills,41
Adelaide Plains,1
Agrelo,2
Alella,1
...,...
Yolo County,1
York Mountain,1
Yorkville Highlands,4


<font color=#a83969>There are numerous categories of regions for a single wine variety, such as 378 for Chardonnay out of a total of 11753, which is an average of 29 regions per variety. This can negatively impact the model's accuracy, so I am eliminating this factor as well.</font>

In [14]:
df=df.drop(['region_1'],axis=1)
df.head()

,country,price,province,winery,variety
0,Italy,23.0,Sicily & Sardinia,Nicosia,White Blend
1,Portugal,15.0,Douro,Quinta dos Avidagos,Portuguese Red
2,US,14.0,Oregon,Rainstorm,Pinot Gris
3,US,13.0,Michigan,St. Julian,Riesling
4,US,65.0,Oregon,Sweet Cheeks,Pinot Noir


In [15]:
df.isnull().sum()

country     39
price        0
province    39
winery       0
variety      0
dtype: int64

<font color=#a83969>After performing all the aforementioned data preprocessing steps, only 39 rows were left empty because of missing values in the 'country' and 'province' columns. These rows were dropped from the dataset as they constitute a very small percentage compared to the size of the original dataset.</font>

In [16]:
df=df.dropna()
df.isnull().sum()

country     0
price       0
province    0
winery      0
variety     0
dtype: int64

<font color=#a83969>After sorting the data and dropping empty rows due to various factors, we have a total of 103283 rows in the dataset, which is only 39 rows less than the original size of 103322 rows.</font>

In [17]:
df.shape

(103283, 5)

In [18]:
df['country'].unique()

array(['Italy', 'Portugal', 'US', 'France', 'Germany', 'Argentina',
       'Chile', 'Australia', 'Austria', 'South Africa', 'New Zealand',
       'Israel', 'Spain', 'Romania', 'Greece', 'Mexico', 'Hungary',
       'Slovenia', 'Luxembourg', 'England', 'Uruguay', 'Lebanon',
       'Canada', 'Brazil', 'Morocco', 'Czech Republic', 'Bulgaria',
       'Cyprus', 'Turkey', 'Moldova', 'Croatia', 'Peru', 'Georgia',
       'Ukraine', 'Switzerland', 'Slovakia', 'Serbia', 'India',
       'Macedonia', 'Armenia'], dtype=object)

In [19]:
df['province'].unique()

array(['Sicily & Sardinia', 'Douro', 'Oregon', 'Michigan', 'Alsace',
       'Rheinhessen', 'California', 'Mosel', 'Other', 'Mendoza Province',
       'Virginia', 'Beaujolais', 'Maule Valley', 'Bordeaux',
       'Maipo Valley', 'Washington', 'Central Italy', 'Champagne',
       'Burgundy', 'South Australia', 'Burgenland', 'New York', 'Tuscany',
       'Leyda Valley', 'Piedmont', 'Stellenbosch',
       'Simonsberg-Stellenbosch', 'Alentejano', 'Southwest France',
       'Aconcagua Valley', 'Loncomilla Valley', 'Marlborough',
       'Colchagua Valley', 'Casablanca Valley', 'Southern Italy',
       'Veneto', 'Northeastern Italy', 'Judean Hills', 'Alentejo',
       'Coastal Region', 'Rhône Valley', 'Western Cape', 'Galilee',
       'Beira Atlantico', 'Northern Spain', 'Kremstal', 'Recas',
       'Santorini', "Hawke's Bay", 'Curicó Valley', 'Limarí Valley',
       'Colchagua Costa', 'Languedoc-Roussillon', 'Provence', 'Catalonia',
       'Rapel Valley', 'Rheingau', 'Naoussa', 'Valle de Guadal

In [65]:
df['winery'].unique()

array(['Nicosia', 'Quinta dos Avidagos', 'Rainstorm', ...,
       'Cardinal Rule', 'Mas de Pampelonne', 'Penedo Borges'],
      dtype=object)

In [66]:
province_stats=df.groupby('province')['province'].agg('count').sort_values(ascending=False)
province_stats.head

<bound method NDFrame.head of province
California          32061
Washington           7579
Bordeaux             5920
Oregon               4917
Tuscany              4785
                    ...  
Viile Timis             1
Szekszárd               1
Lutzville Valley        1
Vinho Espumante         1
Hawaii                  1
Name: province, Length: 370, dtype: int64>

In [67]:
provinces_less_than_100 = province_stats[province_stats >50].index.tolist()
count_provinces_less_than_100 = len(provinces_less_than_100)
count_provinces_less_than_100

91

In [68]:
province_stats_less_than_50=province_stats[province_stats<=50]
province_stats_less_than_50

province
Wagram-Donauland    50
Paarl               47
Chile               46
Elgin               45
Texas               44
                    ..
Viile Timis          1
Szekszárd            1
Lutzville Valley     1
Vinho Espumante      1
Hawaii               1
Name: province, Length: 279, dtype: int64

<font color=#a83969>Currently, there are several provinces in the dataset with very low counts, even some with only one count. Specifically, there are 278 provinces with counts less than 50, 299 provinces with counts less than 100, and 91 provinces with counts greater than 50. Therefore, to simplify the data, I will label all these provinces as "others." As a result, there are now a total of 92 types of provinces, including "others."</font>

In [69]:
df.province=df.province.apply(lambda x: 'other' if x in province_stats_less_than_50 else x)
len(df.province.unique())

92

In [70]:
df.head()

,country,price,province,winery,variety
0,Italy,23.0,Sicily & Sardinia,Nicosia,White Blend
1,Portugal,15.0,Douro,Quinta dos Avidagos,Portuguese Red
2,US,14.0,Oregon,Rainstorm,Pinot Gris
3,US,13.0,Michigan,St. Julian,Riesling
4,US,65.0,Oregon,Sweet Cheeks,Pinot Noir


In [71]:
winery_stats=df.groupby('winery')['winery'].agg('count').sort_values(ascending=False)
winery_stats.head

<bound method NDFrame.head of winery
Testarossa                   217
Williams Selyem              203
Louis Latour                 199
Georges Duboeuf              195
Chateau Ste. Michelle        187
                            ... 
Left Foot Charley              1
Château Respide-Médeville      1
Ruggeri & C.                   1
Rui Reguinga                   1
Hamilton's Ewell Vineyard      1
Name: winery, Length: 14718, dtype: int64>

In [72]:
winery_less_than_10 = winery_stats[winery_stats <=10].index.tolist()
count_winery_less_than_10 = len(winery_less_than_10)
count_winery_less_than_10

12151

<font color=#a83969>In a similar fashion, for winery, there exist 12151 wineries with less than 10 counts. To tackle this, we are marking them as "other". Thus, we have 92 unique wineries, including "other".</font>

In [73]:
winery_stats_less_than_10=winery_stats[winery_stats<=10]
df.winery=df.winery.apply(lambda x: 'other' if x in winery_stats_less_than_10 else x)
len(df.province.unique())

92

<font color=#a83969>To avoid increasing the dimensions significantly and making the model more complex and slow, I opted for factorization method instead of one hot encoding. This method assigned a unique number to each country, province, winery, and variety.</font>

In [74]:
df['variety_number'] = df['variety'].factorize()[0]
df.head()

,country,price,province,winery,variety,variety_number
0,Italy,23.0,Sicily & Sardinia,other,White Blend,0
1,Portugal,15.0,Douro,other,Portuguese Red,1
2,US,14.0,Oregon,Rainstorm,Pinot Gris,2
3,US,13.0,Michigan,other,Riesling,3
4,US,65.0,Oregon,Sweet Cheeks,Pinot Noir,4


In [75]:
df['country'] = df['country'].factorize()[0]
df.head()

,country,price,province,winery,variety,variety_number
0,0,23.0,Sicily & Sardinia,other,White Blend,0
1,1,15.0,Douro,other,Portuguese Red,1
2,2,14.0,Oregon,Rainstorm,Pinot Gris,2
3,2,13.0,Michigan,other,Riesling,3
4,2,65.0,Oregon,Sweet Cheeks,Pinot Noir,4


In [76]:
df['province'] = df['province'].factorize()[0]
df.head()

,country,price,province,winery,variety,variety_number
0,0,23.0,0,other,White Blend,0
1,1,15.0,1,other,Portuguese Red,1
2,2,14.0,2,Rainstorm,Pinot Gris,2
3,2,13.0,3,other,Riesling,3
4,2,65.0,2,Sweet Cheeks,Pinot Noir,4


In [77]:
df['winery'] = df['winery'].factorize()[0]
df.head()

,country,price,province,winery,variety,variety_number
0,0,23.0,0,0,White Blend,0
1,1,15.0,1,0,Portuguese Red,1
2,2,14.0,2,1,Pinot Gris,2
3,2,13.0,3,0,Riesling,3
4,2,65.0,2,2,Pinot Noir,4


In [78]:
df=df.drop(['variety'],axis='columns')
df.head()

,country,price,province,winery,variety_number
0,0,23.0,0,0,0
1,1,15.0,1,0,1
2,2,14.0,2,1,2
3,2,13.0,3,0,3
4,2,65.0,2,2,4


In [79]:
X=df.drop('variety_number',axis=1)
X.head()

,country,price,province,winery
0,0,23.0,0,0
1,1,15.0,1,0
2,2,14.0,2,1
3,2,13.0,3,0
4,2,65.0,2,2


In [81]:
y=df.variety_number
y.head(10)

0    0
1    1
2    2
3    3
4    4
5    5
6    5
7    2
8    6
9    5
Name: variety_number, dtype: int64

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10) 

In [94]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.04346536121666744

In [85]:
from sklearn.linear_model import LogisticRegression
Lr_clf=LogisticRegression()
Lr_clf.fit(X_train,y_train)
Lr_clf.score(X_test,y_test)

C:\Users\saran\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.1580578012296074

In [99]:
from sklearn import tree
dt_clf = tree.DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)
dt_clf.score(X_test,y_test)

0.560342740959481

In [87]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(X_train,y_train)
svc_clf.score(X_test,y_test)

0.16057510771167158

In [96]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier(n_estimators=20)
rfc_clf.fit(X_train, y_train)
rfc_clf.score(X_test,y_test)

0.5616013942005131

In [89]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
gnb_clf.score(X_test,y_test)

0.17311323038195284

In [91]:
from sklearn.naive_bayes import MultinomialNB
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train,y_train)
mnb_clf.score(X_test,y_test)

0.07774604250375175

In [97]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=10)
knn_clf.fit(X_train,y_train)
knn_clf.score(X_test,y_test)

C:\Users\saran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.4670087621629472

In [107]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import tree
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

cross_val_score(tree.DecisionTreeClassifier(), X, y, cv=cv)

array([0.5680883 , 0.56106889, 0.56005228, 0.55961659, 0.56155298,
       0.56344096, 0.55845476, 0.56528053, 0.56334414, 0.55695406])

In [112]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import tree
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=cv)

array([0.57079924, 0.56077843, 0.56160139, 0.56106889, 0.56431234,
       0.56503849, 0.56068161, 0.56455439, 0.56305369, 0.55593745])

<font color=#a83969>The random forest classifier achieved the highest accuracy of 56.27% using 10 k cross validation.</font>

In [114]:
import pickle
with open('wine_variety_predict.pickle','wb') as f:
    pickle.dump(rfc_clf,f)